# Estandarización de Datos del Dataset IGAC

Este notebook aplica estandarización y normalización a los datos limpios.

## Objetivos
1. Ajustar valores monetarios por inflación (IPC)
2. Normalizar códigos DIVIPOLA
3. Crear campos temporales derivados
4. Calcular métricas adicionales
5. Guardar dataset estandarizado

In [1]:
# Importar librerías
import sys
sys.path.append('../src')

import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from etl.data_loader import load_from_parquet, save_to_parquet
from etl.normalizer import apply_all_standardization
from utils.config import CLEANED_PARQUET_FILE, STANDARDIZED_PARQUET_FILE, IPC_DATA, BASE_YEAR

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Cargar Datos Limpios

In [2]:
# Cargar datos limpios desde Parquet
df_cleaned = load_from_parquet(CLEANED_PARQUET_FILE, use_polars=True)
print(f"Datos cargados: {df_cleaned.shape[0]:,} filas, {df_cleaned.shape[1]} columnas")

Cargando datos desde Parquet: d:\conscursoOpendata\igac-anomalias-inmobiliarias\notebooks\..\data\processed\igac_cleaned.parquet
✓ Datos cargados: 30903244 filas, 25 columnas
Datos cargados: 30,903,244 filas, 25 columnas


## 2. Verificar Datos del IPC

In [3]:
df_cleaned.head()

PK,MATRICULA,FECHA_RADICA_TEXTO,YEAR_RADICA,ORIP,DIVIPOLA,DEPARTAMENTO,MUNICIPIO,TIPO_PREDIO_ZONA,CATEGORIA_RURALIDAD,NUM_ANOTACION,ESTADO_FOLIO,FOLIOS_DERIVADOS,Dinámica_Inmobiliaria,COD_NATUJUR,NOMBRE_NATUJUR,NUMERO_CATASTRAL,NUMERO_CATASTRAL_ANTIGUO,DOCUMENTO_JUSTIFICATIVO,COUNT_A,COUNT_DE,PREDIOS_NUEVOS,TIENE_VALOR,TIENE_MAS_DE_UN_VALOR,VALOR
str,str,str,f64,str,str,str,str,str,str,i64,str,str,i64,i64,str,str,str,str,f64,f64,f64,f64,f64,f64
"""05615-020-048347-00006-00842-2…","""020-48347""","""08/02/2022""",2022.0,"""20""","""5615""","""ANTIOQUIA""","""RIONEGRO""","""URBANO""","""CIUDADES Y AGLOMERACIONES""",6,"""INACTIVO""","""SIN_FOLIO""",0,842,"""CANCELACIÓN PROVIDENCIA ADMINI…","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""04-OFICIO 1050-194 DE 02-02-20…",0.0,1.0,0.0,0.0,0.0,0.0
"""05585-019-014037-00005-00427-2…","""019-14037""","""26/01/2017""",2017.0,"""19""","""5585""","""ANTIOQUIA""","""PUERTO NARE""","""URBANO""","""RURAL""",5,"""ACTIVO""","""SIN_FOLIO""",0,427,"""EMBARGO EJECUTIVO CON ACCIÓN P…","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""00-OFICIO 1233 DE 13/12/2016; …",1.0,1.0,0.0,1.0,0.0,0.0
"""11001-50C-2037962-00004-00125-…","""50C-2037962""","""14/01/2019""",2019.0,"""SIN_ORIP""","""11001""","""BOGOTÁ D.C.""","""BOGOTÁ""","""URBANO""","""CIUDADES Y AGLOMERACIONES""",4,"""INACTIVO""","""SIN_FOLIO""",1,125,"""COMPRAVENTA""","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""01-ESCRITURA 4128 DE 17-12-201…",1.0,1.0,0.0,0.0,0.0,0.0
"""25386-166-060114-00007-00900-2…","""166-60114""","""30/07/2015""",2015.0,"""166""","""25386""","""CUNDINAMARCA""","""LA MESA""","""RURAL""","""INTERMEDIO""",7,"""ACTIVO""","""SIN_FOLIO""",0,900,"""OTRO / OFERTA DE COMPRA""","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""00-OFICIO 201552122533 DE 22/0…",0.0,1.0,0.0,1.0,0.0,0.0
"""47245-224-021752-00003-00186-2…","""224-21752""","""22/07/0022""",2022.0,"""224""","""47245""","""MAGDALENA""","""EL BANCO""","""URBANO""","""INTERMEDIO""",3,"""ACTIVO""","""SIN_FOLIO""",1,186,"""TRANSFERENCIA A TÍTULO DE SUBS…","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""00-ESCRITURA 2087 DEL 01/07/22…",1.0,1.0,0.0,1.0,0.0,0.0


In [4]:
# Mostrar datos del IPC que se usarán para ajuste
print("\nÍndice de Precios al Consumidor (IPC) por año:")
print(f"Año base para ajuste: {BASE_YEAR}\n")
for year, ipc in sorted(IPC_DATA.items()):
    print(f"  {year}: {ipc}")


Índice de Precios al Consumidor (IPC) por año:
Año base para ajuste: 2024

  2015: 100.0
  2016: 107.5
  2017: 111.8
  2018: 115.3
  2019: 119.4
  2020: 121.5
  2021: 126.9
  2022: 139.4
  2023: 151.2
  2024: 160.0
  2025: 168.0


## 3. Aplicar Estandarización

In [27]:
import inspect
import etl.normalizer as norm
print(inspect.getsource(norm.apply_all_standardization))


def create_geographic_key(df: pl.DataFrame | pd.DataFrame) -> pl.DataFrame | pd.DataFrame:
    """
    Crea una clave geográfica única combinando departamento y municipio.
    
    Args:
        df: DataFrame
        
    Returns:
        DataFrame con columna GEO_KEY
    """
    print("Creando clave geográfica...")
    
    if isinstance(df, pl.DataFrame):
        df = df.with_columns([
            (pl.col("DEPARTAMENTO") + "_" + pl.col("MUNICIPIO")).alias("GEO_KEY")
        ])
    else:
        df['GEO_KEY'] = df['DEPARTAMENTO'] + "_" + df['MUNICIPIO']
    
    print("✓ Clave geográfica creada")
    return df



In [7]:
# Aplicar todas las funciones de estandarización
df_standardized = apply_all_standardization(df_cleaned)
print(f"\nDataset estandarizado: {df_standardized.shape[0]:,} filas, {df_standardized.shape[1]} columnas")


INICIANDO ESTANDARIZACIÓN DE DATOS

Ajustando valores por inflación (año base: 2024)...
✓ Valores ajustados por inflación
Creando clave geográfica...
✓ Clave geográfica creada

ESTANDARIZACIÓN FINALIZADA


Dataset estandarizado: 30,903,244 filas, 27 columnas


## 4. Verificar Ajuste por Inflación

In [6]:
# Comparar valores originales vs ajustados
comparison = df_standardized.select([
    'YEAR_RADICA',
    'VALOR',
    'VALOR_AJUSTADO'
]).head(10)

print("\nComparación de valores originales vs ajustados:")
print(comparison)


Comparación de valores originales vs ajustados:
shape: (10, 3)
┌─────────────┬───────┬────────────────┐
│ YEAR_RADICA ┆ VALOR ┆ VALOR_AJUSTADO │
│ ---         ┆ ---   ┆ ---            │
│ f64         ┆ f64   ┆ f64            │
╞═════════════╪═══════╪════════════════╡
│ 2022.0      ┆ 0.0   ┆ 0.0            │
│ 2023.0      ┆ 0.0   ┆ 0.0            │
│ 2018.0      ┆ 0.0   ┆ 0.0            │
│ 2021.0      ┆ 0.0   ┆ 0.0            │
│ 2023.0      ┆ 0.0   ┆ 0.0            │
│ 2018.0      ┆ 0.0   ┆ 0.0            │
│ 2021.0      ┆ 0.0   ┆ 0.0            │
│ 2018.0      ┆ 0.0   ┆ 0.0            │
│ 2023.0      ┆ 0.0   ┆ 0.0            │
│ 2018.0      ┆ 0.0   ┆ 0.0            │
└─────────────┴───────┴────────────────┘


In [ ]:
# Estadísticas de valores ajustados por año
valor_stats = df_standardized.group_by('YEAR_RADICA').agg([
    pl.col('VALOR').mean().alias('Valor_Original_Promedio'),
    pl.col('VALOR_AJUSTADO').mean().alias('Valor_Ajustado_Promedio')
]).sort('YEAR_RADICA')

print("\nPromedio de valores por año (original vs ajustado):")
print(valor_stats)

In [ ]:
# Visualizar impacto del ajuste por inflación
plt.figure(figsize=(14, 6))

years = valor_stats['YEAR_RADICA'].to_list()
original = valor_stats['Valor_Original_Promedio'].to_list()
adjusted = valor_stats['Valor_Ajustado_Promedio'].to_list()

x = range(len(years))
width = 0.35

plt.bar([i - width/2 for i in x], original, width, label='Valor Original', alpha=0.8)
plt.bar([i + width/2 for i in x], adjusted, width, label='Valor Ajustado', alpha=0.8)

plt.xlabel('Año')
plt.ylabel('Valor Promedio (COP)')
plt.title(f'Impacto del Ajuste por Inflación (Año Base: {BASE_YEAR})')
plt.xticks(x, years, rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

## 5. Verificar Campos Temporales Derivados

In [ ]:
# Ver campos temporales creados
temporal_fields = df_standardized.select([
    'FECHA_RADICA_TEXTO',
    'YEAR_RADICA',
    'MES_RADICA',
    'TRIMESTRE_RADICA',
    'SEMESTRE_RADICA',
    'DIA_SEMANA_RADICA'
]).head(10)

print("\nCampos temporales derivados:")
print(temporal_fields)

In [ ]:
# Distribución de transacciones por mes
month_dist = df_standardized.group_by('MES_RADICA').agg([
    pl.count().alias('Transacciones')
]).sort('MES_RADICA')

print("\nDistribución de transacciones por mes:")
print(month_dist)

## 6. Verificar Clave Geográfica

In [ ]:
# Ver ejemplos de claves geográficas
geo_sample = df_standardized.select([
    'DEPARTAMENTO',
    'MUNICIPIO',
    'GEO_KEY',
    'DIVIPOLA'
]).unique().head(20)

print("\nEjemplos de claves geográficas:")
print(geo_sample)

## 7. Guardar Datos Estandarizados

In [ ]:
# Guardar dataset estandarizado
save_to_parquet(df_standardized, STANDARDIZED_PARQUET_FILE, compression='snappy')
print(f"\n✓ Datos estandarizados guardados en: {STANDARDIZED_PARQUET_FILE}")

## 8. Resumen de Columnas Finales

In [ ]:
# Listar todas las columnas del dataset final
print("\nColumnas del dataset estandarizado:")
print(f"Total: {len(df_standardized.columns)} columnas\n")
for i, col in enumerate(df_standardized.columns, 1):
    print(f"  {i:2d}. {col}")

## Conclusiones

- ✅ Valores monetarios ajustados por inflación
- ✅ Códigos DIVIPOLA normalizados
- ✅ Campos temporales derivados creados
- ✅ Clave geográfica generada
- ✅ Métricas adicionales calculadas
- ✅ Dataset estandarizado guardado en Parquet

**Próximo paso:** Notebook 04 - Validación de calidad de datos